In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re

# Чтение номеров из файла Excel

In [2]:
numbers_df = pd.read_excel(open('NumberZakupkiGOV.xlsx','rb'))
numbers_list = numbers_df['Номер закупки'].tolist()

In [3]:
session = requests.session()
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/54.0.2810.1 Safari/537.36'}

Функция получение текста HTML-файла

In [4]:
def get_html(url):
    r = session.get(url, headers=headers)
    return r.text

In [5]:
numbers = []
starting_price = []
customer = []
purchase_object = []
redaction = []
documentation = []
place = []
index = []

Программа будет брать данные с 3 разных страниц: страница результата поиска по номеру, страница общей информации по закупке и страница документации к ней. Для этого разделим код на 3 части.

# Выгрузка данных

### 1. Страница результатов поиска по номеру

Url будет состоять из двух частей, между которыми будет вставляться номер закупки

In [6]:
url1 = 'https://zakupki.gov.ru/epz/order/extendedsearch/results.html?searchString='
url2 = '&morphology=on&search-filter=Дате+размещения&pageNumber=1&sortDirection=false&recordsPerPage=_10&showLotsInfoHidden=false&sortBy=UPDATE_DATE&fz44=on&fz223=on&af=on&ca=on&pc=on&pa=on&currencyIdGeneral=-1'

Функция получения цены. В случае если цена на сайте отсутствует, возвращается пустое значение

In [7]:
def get_price(html):
    soup = BeautifulSoup(html, 'lxml')
    try:
        price = soup.find('form', class_='formValidation search search-quick').find_all('div', class_='container')[1].find('div', class_='row').find('div', class_='col-9 search-results').find('div', class_='search-registry-entrys-block').find('div', class_='search-registry-entry-block box-shadow-search-input').find('div', class_='row no-gutters registry-entry__form mr-0').find('div', class_='col col d-flex flex-column registry-entry__right-block b-left').find('div', class_='price-block').find('div', class_='price-block__value').text
        price = price.strip()
        return re.sub('\xa0', '', price)
    except:
        return ''

In [8]:
for number in numbers_list:
    url = url1 + str(number) + url2
    html = get_html(url)
    starting_price.append(get_price(html))

### 2. Страница с общей информацией

In [9]:
const_url = 'https://zakupki.gov.ru/223/purchase/public/purchase/info/common-info.html?regNumber='

Функция получения номера закупки, заказчика, объекта закупки и редакции

In [10]:
def get_data(html):
    data = []
    soup = BeautifulSoup(html, 'lxml')
    path = soup.find('div', class_='cardWrapper').find('div', class_='wrapper').find('div', class_='mainbox').find('div', class_='cardWrapper').find('div', class_='wrapper').find('div', class_='mainBox').find('div', class_='contentTabBoxBlock').find('div', class_='noticeTabBox padBtm20')
    path1 = path.find('div', class_='noticeTabBoxWrapper')
    path2 = path.find_all('div', class_='noticeTabBoxWrapper')[1]
    data.append(path1.find_all('td')[3].text.strip()) #номер закупки
    data.append(path2.find_all('td')[1].text.strip()) #заказчик
    data.append(path1.find_all('td')[10].text.strip()) #редакция
    data.append(path1.find_all('td')[7].text.strip()) #объект закупки 
    return data

Функция получения места подведения итогов

In [11]:
def get_place(html):
    soup = BeautifulSoup(html, 'lxml')
    
    place = soup.find('div', class_='cardWrapper').find('div', class_='wrapper').find('div', class_='mainbox').find('div', class_='cardWrapper').find('div', class_='wrapper').find('div', class_='mainBox').find('div', class_='contentTabBoxBlock').find('div', class_='noticeTabBox padBtm20')
    for i in range(0, len(place.find_all('h2'))):
        if place.find_all('h2')[i].text == 'Порядок проведения процедуры':
            place = place.find_all('div', class_='noticeTabBoxWrapper')[i]
            for j in range(0, len(place.find_all('td'))):
                if place.find_all('td')[j].text == 'Место подведения итогов':
                    return place.find_all('td')[j+1].text

In [12]:
for number in numbers_list:
    url = const_url + str(number)
    html = get_html(url)
    data = get_data(html)
    numbers.append(data[0])
    customer.append(data[1])
    redaction.append(data[2])
    purchase_object.append(data[3])
    place.append(get_place(html))

Получение индекса из места подведения итогов с помощью регулярного выражения

In [13]:
import re
for i in range(len(place)):
    parser = re.findall('(\d{6})', place[i])
    try:
        index.append(int(parser[0]))
    except:
        index.append('')

### 3. Страница с документацией

In [14]:
url_doc ='https://zakupki.gov.ru/223/purchase/public/purchase/info/documents.html?regNumber='

In [15]:
def get_documentation(html):
    soup = BeautifulSoup(html, 'lxml')
    
    documentation =soup.find('div', class_='cardWrapper').find('div', class_='wrapper').find('div', class_='mainbox').find('div', class_='cardWrapper').find('div', class_='wrapper').find('div', class_='mainBox').find('div', class_='contentTabBoxBlock').find('div', class_='noticeTabBox padBtm20').find('div', class_='contentTabBoxBlock').find('div', class_='noticeTabBox').find('div', class_='noticeTabBoxWrapper').find('div', class_='addingTbl padTop10 padBtm10 autoTh')
    documentation = documentation.find('table', class_='documentTablet').find('tr', class_='odd').find_all('a')[1].text
    return documentation

In [16]:
for number in numbers_list:
    url = url_doc + str(number) 
    html = get_html(url)
    documentation.append(get_documentation(html))

# Составление таблицы

In [17]:
comp_df = pd.DataFrame(
    {  
        'Номер закупки': numbers,
        'Начальная цена': starting_price,
        'Заказчик': customer,
        'Объект закупки': purchase_object,
        'Редакция' : redaction,
        'Документ по закупке' : documentation,
        'Место подведения итогов': place,
        'Индекс' : index
    })
comp_df

,Номер закупки,Начальная цена,Заказчик,Объект закупки,Редакция,Документ по закупке,Место подведения итогов,Индекс
0,32009401407,"60830000,00 ₽","ПУБЛИЧНОЕ АКЦИОНЕРНОЕ ОБЩЕСТВО ""СБЕРБАНК РОССИИ""",ЗП - 81031191 (проектирование и строительство ...,1,Документация аукциона.docx,"117997, Москва, ул. Вавилова, 19",117997
1,32009417101,"4285524,88 ₽","ПУБЛИЧНОЕ АКЦИОНЕРНОЕ ОБЩЕСТВО ""СБЕРБАНК РОССИИ""",ЗП – 81031710 выполнение работ по перепланиров...,1,ЗП81031710.docx,"443077, г. Самара, Московское шоссе, 15, Повол...",443077
2,32009440741,,"ПУБЛИЧНОЕ АКЦИОНЕРНОЕ ОБЩЕСТВО ""СБЕРБАНК РОССИИ""",ЗП – 81014837 ОКАЗАНИЕ УСЛУГ ПО ОЦЕНКЕ ИМУЩЕСТ...,1,Документация.docx,"620026, г. Екатеринбург, ул. Куйбышева, 67",620026
3,32009457505,"1737552,30 ₽","ПУБЛИЧНОЕ АКЦИОНЕРНОЕ ОБЩЕСТВО ""СБЕРБАНК РОССИИ""",ЗП-81032015 (Выполнение работ по реконструкции),1,ЮЗБ2020(ЗКА.) (1).docx,"Юго-Западный банк ПАО Сбербанк:344068, г. Рост...",344068
4,32009457949,"11359572,00 ₽","ПУБЛИЧНОЕ АКЦИОНЕРНОЕ ОБЩЕСТВО ""СБЕРБАНК РОССИИ""",ЗП – 81031774 ВЫПОЛНЕНИЕ СТРОИТЕЛЬНО-МОНТАЖНЫХ...,1,ЗП 81031774.docx,"г. Воронеж, ул. Плехановская, д. 48Б.",
5,32009465565,"1794895,00 ₽","ПУБЛИЧНОЕ АКЦИОНЕРНОЕ ОБЩЕСТВО ""СБЕРБАНК РОССИИ""",ЗП-81032406 (строительно-монтажные работы на о...,1,ЗК обособление Большевиствская 30.docx,"Волго-Вятский банк ПАО Сбербанк (603005, г. Н....",603005
6,32009475339,"2467999,97 ₽","ПУБЛИЧНОЕ АКЦИОНЕРНОЕ ОБЩЕСТВО ""СБЕРБАНК РОССИИ""",ЗП - 81031715 (ВЫПОЛНЕНИЕ СТРОИТЕЛЬНО -МОНТАЖ...,1,Документация (5).docx,"Байкальский банк ПАО Сбербанк 664011, г. Иркут...",664011
7,32009475352,"7200000,00 ₽","ПУБЛИЧНОЕ АКЦИОНЕРНОЕ ОБЩЕСТВО ""СБЕРБАНК РОССИИ""",ЗП–81031805 МОНТАЖ СИСТЕМ БЕЗОПАСНОСТИ,1,02 ЗП-81031805 (АСМСП) v1.docx,"г. Санкт-Петербург, ул. Красного Текстильщика,...",
8,32009480108,"4775000,00 ₽","ПУБЛИЧНОЕ АКЦИОНЕРНОЕ ОБЩЕСТВО ""СБЕРБАНК РОССИИ""",ЗП – 81032320 (Выполнение работ по разработке ...,1,Документация 65-3.docx,"620026, г. Екатеринбург, ул. Куйбышева, 67",620026
9,32009483456,"999997,67 ₽","ПУБЛИЧНОЕ АКЦИОНЕРНОЕ ОБЩЕСТВО ""СБЕРБАНК РОССИИ""",ЗП – 81032549 (капитальный ремонт объекта),1,ЗК кап.рем. Глазов разм.docx,"603005, Россия, Нижегородская область, г. Нижн...",603005


### Доля закупок, которые относятся к стройтельно-монтажным работам 33%

In [18]:
count = 0
for i in range(comp_df.shape[0]):
    if re.search('(СТРОИТЕЛЬНО.*?МОНТАЖНЫХ)', comp_df.iloc[i, 3]) != None:
        count += 1
proportion = count/comp_df.shape[0]
proportion

0.3333333333333333

### Количество строк в правильном склонении

In [19]:
count_of_strings = comp_df.shape[0]
remainder = count_of_strings % 10
if count_of_strings == 0:
    print('В таблице нет строк')
elif remainder == 0 or remainder >=5 or (count_of_strings >=11 and count_of_strings <=19):  
    print('В таблице', count_of_strings, 'строк')
elif remainder == 1:  
    print('В таблице', count_of_strings, 'строка')
else:
    print('В таблице', count_of_strings, 'строки')

В таблице 15 строк


### Подсчитать сумму по столбцу «Начальная цена»

In [20]:
summary = 0
for i in range(comp_df.shape[0]):
    if comp_df.iloc[i, 1] != '':
        temp = float(re.sub(',','.',re.sub(' ₽', '', comp_df.iloc[i, 1])))
        summary += temp
print(str(round(summary, 2)) + ' ₽')

124448973.82 ₽


# Сохранение в файл MS Excel

In [21]:
comp_df.to_excel("result.xlsx", index=False)